In [230]:
from pgmpy.readwrite import BIFReader
import random
import numpy

In [231]:
#Parameters
GROUP_ID = '04'
ALGORITHM = 'gibbs'
NETWORK_NAME = 'networks/child.bif'
REPORT = 'Disease'
EVIDENCE_LEVEL = 'None'
EVIDENCE = 'owerBodyO2="<5"; RUQO2="12+"; CO2Report=">=7.5"; XrayReport=Asy/Patchy'

In [232]:
def normalizeDistribution(distribution):
    sum = 0
    for value in distribution:
        sum += value
    for i in range(len(distribution)):
        distribution[i] = distribution[i] / sum
    return distribution

In [233]:
def gibbsSampling(Network, reportedVars, evidenceVars, numSamples, burnInLength):
    possVals = Network.states
    randomVals = {}

    #Initializing random state values
    for key in possVals.keys():
        if key in evidenceVars.keys():
            randomVals[key] = evidenceVars[key]
        else:
            randomVals[key] = random.choice(possVals[key])

    #initializing counting table
    countingTable = []
    for i in range(len(possVals[reportedVars[0]])):
        countingTable.append(0)

    for i in range(numSamples):
        #Need to pick a random (non evidence) node use its markov blanket
        randVar = random.choice(list(Network.nodes))
        while(randVar in evidenceVars.keys()):
            randVar = random.choice(list(Network.nodes))
        mb = getParentsAndChildren(Network, randVar)
        #Calculate probability of variable given its parents
        cpds = Network.get_cpds(randVar)
        valueDistribution = []
        for value in cpds.state_names[randVar]:
            #Calculates P(x_i|parents(X_i))
            parents = mb[0]
            valueArrIndex = cpds.name_to_no[randVar][value]
            valueArr = cpds.values[valueArrIndex]
            for parent in parents:
                parentVal = randomVals[parent]
                parentArrIndex = cpds.name_to_no[parent][parentVal]
                valueArr = valueArr[parentArrIndex]
            varVal = valueArr
            childrenProb = 1
            #Calculates/sums all child probabilities P(y_j | parents(y_j))
            for children in mb[1]:
                childCpds = Network.get_cpds(children)
                parents = Network.get_parents(children)
                childrenArrIndex = childCpds.name_to_no[children][randomVals[children]]
                childrenArr = childCpds.values[childrenArrIndex]
                for parent in parents:
                    if(parent != randVar):
                        parentVal = randomVals[parent]
                    else:
                        parentVal = value
                    parentArrIndex = childCpds.name_to_no[parent][parentVal]
                    childrenArr = childrenArr[parentArrIndex]
                childrenProb *= childrenArr
            valueDistribution.append(varVal*childrenProb)
        #normalized value distribution for randomly selected variable has been found
        valueDistribution = normalizeDistribution(valueDistribution)
        #print("Value distribution:", valueDistribution)
        possibleVals = Network.states[randVar]
        #print("Possible values for variable", randVar, possibleVals)
        chosenVal = numpy.random.choice(possibleVals, p = valueDistribution)
        #print("Chosen value:", chosenVal)
        #setting the variable's randomly (probability-distribution) based value
        randomVals[randVar] = chosenVal
        #counting value for reported variable if not in burn in period
        if(i >= burnInLength):
            countingIndex = possVals[reportedVars[0]].index(randomVals[reportedVars[0]])
            countingTable[countingIndex] += 1


    countingTable = normalizeDistribution(countingTable)
    print(countingTable)


def getParentsAndChildren(Network, variable):
    mb = Network.get_markov_blanket(variable)
    nodes = []
    parents = Network.get_parents(variable)
    children = []
    for node in mb:
        nodeParents = Network.get_parents(node)
        if variable in nodeParents:
            children.append(node)
    nodes.append(parents)
    nodes.append(children)
    return nodes

In [234]:
reader = BIFReader(NETWORK_NAME)
model = reader.get_model()
reportVariables = REPORT.split(';')
evidenceVariables = {}
splitter = EVIDENCE.split(";")
for var in splitter:
    #Edge case for variables with '=' characters in their values
    if'"' in var:
        splitter2 = var.split('"')
        evidenceVariables[splitter2[0][:-1].strip()] = splitter2[1]
    else:
        splitter2 = var.split("=")
        evidenceVariables[splitter2[0].strip()] = splitter2[1]

if ALGORITHM == "gibbs":
    gibbsSampling(model,reportVariables,evidenceVariables, 11000, 1000)
elif ALGORITHM == "ve":
    print()
    #variable elim
else:
    print("Unrecognized algorithm:", ALGORITHM)





[0.12, 0.0788, 0.2329, 0.1773, 0.0664, 0.3246]
